In [8]:
import random
import matplotlib.pyplot as plt
import math
from math import comb

In [9]:
# Gives every 0 or 1 (discard or keep) combination for the number of dice we trow.
def dynamic_loop(depth, current_depth=0, current_combination=[], result=None):
    global i
    if result is None:
        result = {}
        i = 0
    
    if current_depth == depth:
        result[i] = current_combination
        i += 1
        return result
    
    #Keep or discard: Where 0 is discard and 1 is keep
    
    for value in [0,1]:
        dynamic_loop(depth, current_depth + 1, current_combination + [value], result)
    
    return result


In [10]:
def xOfAKind(x, keepList, dice):

    #Return 0 if theres less dice available to us, than what we want.
    if len(dice) < x:
        return [0], 0.0
    
    # If a number in keepList is 1, we keep otherwise we HAVE to throw it.
    # Count all number we keep and throw.
    keepDict = {}
    unique = 0
    keepAmount = 0
    throwAmout = 0
    
    for i, keep in enumerate(keepList):
        if keep == 0:
            throwAmout += 1
        else:
            keepAmount += 1
            if dice[i] in keepDict:
                keepDict[dice[i]] += 1
            else:
                keepDict[dice[i]] = 1
                unique += 1

    # Check the keepDict for highest x of a kind.
    tmpK = 0
    for key, value in keepDict.items():
        if value >= x:
            if key > tmpK:
                tmpK = key
    if tmpK != 0: 
        return [tmpK], 1.0
    
    # In case we dont want to keep any dice, for example keepDice = [0,0,0]
    # We shouldnt multiply or device with 0
    if unique == 0:
        return available, ((1/6)**x)

    # Probability of rolling a specific number on a die
    p_outcome = unique / 6
    n_p_outcome = 1-p_outcome

    # n = number of dice we're rolling
    # k = number of additional dice needed to match the value we're aiming for
    n = throwAmout
    k = int(x-(keepAmount/unique))

    results = (comb(n, k))*(p_outcome**k)*(n_p_outcome**(n-k))

    keys = list(keepDict.keys())

    return keys, results

In [11]:
# Calculating the probability of getting num, given that we're keeping the numbers in keepList
def inference(keepList, dice, available):
    inf = 0
    diceCopy = dice.copy()
    keepListCopy = keepList.copy()
    di = 0
    
    # Do do not take dice not in available into concideration
    for i in range(len(dice)):
            if dice[i] not in available and keepList[i] == 1:
                del diceCopy[i]
                del keepListCopy[i]
    
    # Calculate Chance of 1 of a kind, 2 of a kind ... n of a kind and add to inf
    for i in range(1,len(dice)+1):
        d, p = xOfAKind(i, keepListCopy, diceCopy)
        
        # take the average of the numbers returned times with the probability times the number of similar. 
        score = p
        
        # add the score to the total inference
        inf += score 
        
    return inf, keepList, dice

In [12]:
# Reroll all dice that we do not want to keep.
def reroll(dice, keepList):
    diceC = dice.copy()
    for i in range(len(keepList)):
        if keepList[i] == 0:
            diceC[i] = random.randint(1, 6)

    return diceC

In [13]:
from collections import Counter

# Do a starting roll -> the first inference and decision -> the second inference and decision
# Return the dice we kept, and how many we got.

def oneRound(numberOfDice, available):
    # A dict of all combinations
    combinations_dict = dynamic_loop(numberOfDice)
    dice = []
    keepDice = 0
    keepList = []
    topInf = 0

    # Initial roll, roll all num of dice
    for _ in range(numberOfDice):
        dice.append(random.randint(1, 6))
    

    # Custom dice for testing purposes.
    # dice = [2, 2, 2, 2, 5]

    print(f"inital roll : {dice}")

    # Do inference on all combinations of "keep or discard" with the current dice
    for combination in combinations_dict.items():
        inf, kL, d = inference(combination[1], dice, available)
        if inf > topInf:
            keepList = kL
            topInf = inf
    
    
    dice = reroll(dice, keepList)
    print(f"first reroll : {dice}")

    # Do inference on all combinations of "keep or discard" with the current dice
    for combination in combinations_dict.items():
        inf, kL, d = inference(combination[1], dice, available)
        if inf > topInf:
            keepList = kL
            topInf = inf
    dice = reroll(dice, keepList)
    print(f"second reroll : {dice}")

    # Do inference on all combinations of "keep or discard" with the current dice
    for combination in combinations_dict.items():
        inf, kL, d = inference(combination[1], dice, available)
        if inf > topInf:
            keepList = kL
            topInf = inf    

    # Return the most occuring dice
    counter = Counter(d)
    maxOccuringDice, amount = counter.most_common(1)[0]

    return maxOccuringDice, amount


In [14]:
numberOfDice = 5
totalScore = 0
available = [1, 2, 3, 4, 5, 6]

for i in range(6):
    print(f"Round -- {i+1}")
    print(f"available : {available}")
    kept, amount = oneRound(numberOfDice, available)
    print(f"kept {kept}: amount {amount}")
    available.remove(kept)
    totalScore += kept*amount


print(f"points : {totalScore}")


    

Round -- 1
available : [1, 2, 3, 4, 5, 6]
inital roll : [5, 6, 2, 2, 2]
first reroll : [3, 2, 2, 2, 2]
second reroll : [5, 2, 2, 2, 2]
kept 2: amount 4
Round -- 2
available : [1, 3, 4, 5, 6]
inital roll : [6, 5, 6, 4, 4]
first reroll : [5, 2, 6, 4, 4]
second reroll : [6, 4, 6, 4, 4]
kept 4: amount 3
Round -- 3
available : [1, 3, 5, 6]
inital roll : [5, 1, 2, 3, 2]


IndexError: list assignment index out of range